Устанавливаем библиотеки: pandas - для работы с таблицами, pyreadstat - для чтения файлов из статистических программ (типа SPSS). Обязательно указываем версию библиотеки через ==, потому что код, работающий в одной версии, может не работать в другой.  
Документации на них:

https://pandas.pydata.org

https://ofajardo.github.io/pyreadstat_documentation/_build/html/index.html


In [1]:
!pip install pandas==1.4.2 pyreadstat==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Подключаем Google Disk, надо будет дать разрешение этой jupyter-notebook тетрадке на чтение/запись с него. Файлы *.sav должны быть заранее загружены на Google Disk. Если они загружены не в корень диска, а в подпапку, надо будет поменять переменные fpath (cм. ниже)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# данные 2011 года не используем, так как в них нет нужных нам переменных

fpath_2014 = 'drive/MyDrive/коуж/IND_2014.sav'
fpath_2016 = 'drive/MyDrive/коуж/IND_2016.sav'
fpath_2018 = 'drive/MyDrive/коуж/IND_KOUZH_2018.sav' 
fpath_2020 = 'drive/MyDrive/коуж/IND_OSN_2020.sav'

In [4]:
# импортируем библиотеку pyreadstat, 
# которая ориентирована на работу с форматами статистических (в т.ч., платных) программ.
import pyreadstat

In [5]:
df_2014, meta_2014 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2014, num_processes=16, encoding='cp1251') 
df_2016, meta_2016 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2016, num_processes=16, encoding='cp1251') 

In [6]:
# Источник данных: https://gks.ru/ .
df_2018, meta_2018 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2018, num_processes=16, encoding='cp1251') 


In [7]:
df_2020, meta_2020 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2020, num_processes=16, encoding='cp1251') 

In [8]:
# Чтобы выбрать только определенные колонки из dataframe, заводим список их названий.
# подробнее о списках - https://pythontutor.ru/lessons/lists/#section_1
selection = ['I05_04', 'I05_06', 'I05_11', 'I05_16', 'I05_17_02', "H00_02"] # переменные, касающиеся работы + код субъекта РФ

In [9]:
def descriptional_statistics(df, meta, selection):
    # Создадим новую таблицу (pandas DataFrame), но только с теми колонками,
    # которые используются в расчётах.
    # Так используется меньше памяти, удобнее визуализировать. Но можно пользоваться и исходной таблицей.
    selected = df[selection]
    # Используем генератор списков, чтобы переименовать служебные заголвки в читаемые.
    # для этого мы хотим "пройтись" по всем меткам в списке selection и получить новый список вместо selection.
    # Подробнее о генераторах списков: https://pythontutor.ru/lessons/lists/#section_3
    selection_names = [meta.column_names_to_labels[label] for label in selection]
    # Заменим исходные наименования колонок на наименования из получившегося списка selection_names, 
    # теперь у нас наименования колонок будет как в столбце "метки" файла sav.
    selected.columns = selection_names
    # 98 - код Республики Саха (Якутии), согласно ОКВЭДам.
    # создаем маску, чтобы получить таблицу с данными только по Якутии.
    mask = selected['Код субъекта РФ'] == '98'
    # применяем маску - используем только те строчки, где значения маски True
    yakutia = selected[mask]
    # число наблюдений в таблице получаем через атрибут shape.
    # так как в shape находится объект типа кортеж (tuple)- структура данных, похожая на список,
    # но неизменяемая (чтобы добавить или удалить элемент, придется создавать новый объект типа кортеж в памяти).
    # из кортежа,  так же, как из списка, можно получить по порядковому индексу его элемент
    # в данном случае у нас shape[0] - количество строк, shape[1] - количество столбцов.
    # pandas DataFrame - двумерная таблица, ее размерность описывается количеством строчек и столбцов.
    # одномерная "таблица" pandas, по сути, одна колонка, размерность которой описывается только количеством строчек,
    # называется Series

    yakutia.dropna(inplace=True) # исключим наблюдения с пропущенными значениями
    print(f"Количество наблюдений по выбранным переменным: {yakutia.shape[0]}")
    # apply - метод pandas DataFrame, который позволяет применить функцию ко всему Series или DataFrame,
    # не прописывая цикл явно.

    # Применим функцию, который переведет служебные значения в читаемые ко всему столбцу Размер населенного пункта.

    # в python есть два способа объявить функцию - через ключевое слово def или lambda.
    # def удобно использовать, когда у функции много аргументов и/или в ней выполняется много действий
    # lambda удобно использовать, когда функция принимает 1-2 аргумента и выполняемые действия можно прописать в 
    # одну строчку

    # После ключевого слова lambda идет аргумент функции (x), двоеточие и после него указываем прописываем действие 
    # над аргументом х

    # В данном случае необходимо из словаря variable_to_label получить значение ключа R_1_1 
    # (соответствует размеру населённого пункта), а затем по этому значению получить другое значение 
    # из словаря value_labels
    # т.е., value_labels это вложенный словарь - словарь в словаре.
    for name, variable in zip(selection_names[:-1], selection[:-1]):
        possible_answers = meta.value_labels[meta.variable_to_label[variable]]
        variable_values = yakutia[name].apply(lambda x: possible_answers[x])
        print(name)
        print(f"Возможные ответы: {possible_answers}")
        print(variable_values.describe())
        print("--------------------------")

Что считает метод describe для нечисловых данных:

unique - количество уникальных значений

top - наиболее часто встречающееся значение

freq - частота top-а (мода)

в случае нескольких одинаковых top-ов, какой вывести, выбирается случайно

In [10]:
print("2014 ГОД")
descriptional_statistics(df_2014, meta_2014, selection)

2014 ГОД
Количество наблюдений по выбранным переменным: 510
Основная работа
Возможные ответы: {1.0: 'на предприятии, в организации (или обособленном подразделени', 2.0: 'на предприятии индивидуального предпринимателя, семейном пре', 3.0: 'в фермерском хозяйстве', 4.0: 'в сфере предпринимательской деятельности без образования юри', 5.0: 'по найму у физических лиц, индивидуальных предпринимателей', 6.0: 'на индивидуальной основе', 7.0: 'в собственном домашнем хозяйстве по производству товаров, пр'}
count                                                   510
unique                                                    4
top       на предприятии, в организации (или обособленно...
freq                                                    444
Name: Основная работа, dtype: object
--------------------------
Работали в качестве
Возможные ответы: {1.0: 'наемного работника за заработную плату или вознаграждение  д', 2.0: 'ученика на производстве, стажера, практиканта', 3.0: 'владельца (совладельца) со

<ipython-input-9-86e006bd73d7>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yakutia.dropna(inplace=True) # исключим наблюдения с пропущенными значениями


In [11]:
print("2016 ГОД")
descriptional_statistics(df_2016, meta_2016, selection)

2016 ГОД
Количество наблюдений по выбранным переменным: 475
Основная работа
Возможные ответы: {1.0: 'На предприятии, в организации со статусом юридического лица', 2.0: 'На предприятии индивидуального предпринимателя', 3.0: 'В фермерском хозяйстве', 4.0: 'В сфере предпринимательской деятельности без образования юри', 5.0: 'По найму у физических лиц, индивидуальных предпринимателей', 6.0: 'На индивидуальной основе', 7.0: 'В собственном домашнем хозяйстве по производству товаров, пр'}
count                                                   475
unique                                                    4
top       На предприятии, в организации со статусом юрид...
freq                                                    428
Name: Основная работа, dtype: object
--------------------------
Работали в качестве
Возможные ответы: {1.0: 'Наемный работник за заработную плату или вознаграждение день', 2.0: 'Ученик на производстве, стажер, практикант', 3.0: 'Владелец (совладелец) собственного предприят

<ipython-input-9-86e006bd73d7>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yakutia.dropna(inplace=True) # исключим наблюдения с пропущенными значениями


In [12]:
print("2018 ГОД")
descriptional_statistics(df_2018, meta_2018, selection)

2018 ГОД
Количество наблюдений по выбранным переменным: 498
Основная работа
Возможные ответы: {1.0: 'На предприятии, в организации со статусом юридического лица', 2.0: 'На предприятии индивидуального предпринимателя или у лиц, ос', 3.0: 'В фермерском хозяйстве', 4.0: 'В сфере предпринимательской деятельности (в том числе в парт', 5.0: 'По найму в частных домохозяйствах (у частных лиц)', 6.0: 'На индивидуальной основе', 7.0: 'В собственном домашнем хозяйстве по производству товаров для'}
count                                                   498
unique                                                    4
top       На предприятии, в организации со статусом юрид...
freq                                                    437
Name: Основная работа, dtype: object
--------------------------
Работали в качестве
Возможные ответы: {1.0: 'Наемный работник  за заработную плату или вознаграждение  де', 2.0: 'Ученик на производстве, стажер, практикант', 3.0: 'Владелец (совладелец) собственного  пре

<ipython-input-9-86e006bd73d7>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yakutia.dropna(inplace=True) # исключим наблюдения с пропущенными значениями


In [13]:
print("2020 ГОД")
descriptional_statistics(df_2020, meta_2020, selection)

2020 ГОД
Количество наблюдений по выбранным переменным: 546
Основная работа
Возможные ответы: {1.0: 'На предприятии, в организации со статусом юридического лица', 2.0: 'На предприятии индивидуального предпринимателя или у лиц, ос', 3.0: 'В фермерском хозяйстве', 4.0: 'В сфере предпринимательской деятельности (в том числе в парт', 5.0: 'По найму в частных домохозяйствах (у частных лиц)', 6.0: 'На индивидуальной основе', 7.0: 'В собственном домашнем хозяйстве по производству товаров для'}
count                                                   546
unique                                                    4
top       На предприятии, в организации со статусом юрид...
freq                                                    445
Name: Основная работа, dtype: object
--------------------------
Работали в качестве
Возможные ответы: {1.0: 'Наемный работник  за заработную плату или вознаграждение  де', 2.0: 'Ученик на производстве, стажер, практикант', 3.0: 'Владелец (совладелец) собственного  пре

<ipython-input-9-86e006bd73d7>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yakutia.dropna(inplace=True) # исключим наблюдения с пропущенными значениями
